In [1]:
import datetime
import os
import numpy as np
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from obs_data import *

print(os.getcwd())

2016
D:\Documents\these_pablo\Models\BEACH2016\Analysis


### Path and working computer

In [2]:
PC = True
analysis = True
version = "model_v12"
folder = "1"
short_sim = True
timestamp = 'julian'

In [3]:
# Path to simulated folder
if analysis:
    dot = ''
else:
    dot = '..\\'

# Working computer
PC = False
if PC:
    back = "\\"  
    sim_path = version + back  
else:
    back = "/"
    sim_path = version + back
    
print(sim_path)

model_v12/


### Variable and transect 

In [4]:
variable = 'd13C'
transect = 'North'
trans = 'nor'

In [5]:
if variable == 'CONC':
    det_name = "ug.g"
    name = 'Conc.mug.g.dry.soil'
else:  # d13C
    det_name = "det.d13C"  # Source: "Data/BEACH_R/
    name = "comp.d13C"

if transect == 'North':
    detail_plots = ['n1', 'n2', 'n3', 'n4', 'n5', 'n7', 'n8']
    
elif transect == 'Valley':
    detail_plots = ['t4', 't5', 't7', 't8', 't9']
    
else:
    detail_plots = ['s11', 's12', 's13']

palette = ["#F44336", '#673AB7', '#3F51B5', '#009688', '#FF9800', '#4CAF50', '#03A9F4']
# red, purple, indigo, teal-green, orange, green, light-blue

### Data frame, date format and date range

In [6]:
n_obs = getSoilData(transect, tss=True)  # n_obs.head()
nVar = n_obs[name].tolist()

if timestamp == 'julian':
    # Julian days
    x = n_obs.Jdays.tolist()
else:
    # Calendar days
    dates = getTime(sep=";")
    x = dates.Date.tolist()

# Observations 
if short_sim:
    s_obs = 166 - 1
    e_obs = 300
else:
    # Julian days
    s_obs = 0
    e_obs = 300
    
# q_obs[s_obs:(s_obs+20)]

### Observed

In [7]:
data = []

#### Composites

In [8]:
nObs = go.Scatter(
    x = x[s_obs:e_obs],
    y = nVar[s_obs:e_obs],
    mode = 'markers',
    name = transect + ' Composite '  + variable,
    marker=dict(
        color= 'black', # 'rgba(204, 204, 204, 0.95)',
        line=dict(
            color= 'black', #'rgba(217, 217, 217, 1.0)',
            width=1,
        ),
        symbol='circle',
        size=10)
)

data.append(nObs)

#### Detailed

In [9]:
# Append observed detailed
for i in range(len(detail_plots)):
    n = getDetailed(detail_plots[i] + ".tss")
    n = n[det_name].tolist()
    n_det = go.Scatter(
        x = x[s_obs:e_obs],
        y = n[s_obs:e_obs],
        mode = 'markers',
        name = detail_plots[i],
        marker=dict(
            color= palette[i], 
            line=dict(
                color= 'black', #'rgba(217, 217, 217, 1.0)',
                width=1,
            ),
            symbol='star',
            size=10)
    )
    data.append(n_det)

### Simulated

In [10]:
# Append composite simulations
col = 'sim' 
filename = "resM_" + trans + variable + ".tss"
sim_table = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)

sim = sim_table[col].tolist()

trace = go.Scatter(
    x = x[s_obs:e_obs],
    y = sim,
    mode = 'lines',
    name = transect + ' Sim.',
    marker=dict(
        color= 'black')
)
data.append(trace)

In [11]:
# Append detailed simulations
for i in range(len(detail_plots)):
    plot = detail_plots[i]
    filename = "resM_" + plot + variable + ".tss"
    det_sim = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', col],
                                header=None)
    sim = det_sim[col].tolist()

    trace = go.Scatter(
        x = x[s_obs:e_obs],
        y = sim,
        name = plot + ' Sim.',
        line = dict(
            color = palette[i], 
            width = 1,
            dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
    )
    data.append(trace)


In [12]:
%matplotlib inline
import plotlycred
figname = 'Soils' + variable + transect + '.png'

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data) # , layout=layout)
    py.image.save_as(fig, filename=figname)
else:
    iplot(data)


Link:

https://plot.ly/~DayTimeChunks/35/


In [13]:
from IPython.display import Image
if online:
    i = Image(figname)
    display(i)